In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import InputLayer
import pandas as pd
from tensorflow.keras.utils import plot_model
import numpy as np
import pandasql as ps
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
from scipy.stats import chi2_contingency
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split 
#from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingRandomSearchCV 
from sklearn.ensemble import RandomForestClassifier 
import keras_tuner as kt

In [3]:
data= pd.read_csv(r'C:\Users\liatw\OneDrive\Documents\clinical data project\heart_failure_clinical_records_dataset.csv')
data

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [27]:
###########################################################
### Building predictive model using Logistic regression ###
###########################################################

# Build ordinal variables from continous variables by dividing continous variables into quartiles
num_var_list = ['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine',
                'serum_sodium','time'] 
for var in num_var_list:
    #data[var+'_'+'q'] = pd.qcut(data[var],4,labels=False)
    cut_series, cut_intervals = pd.qcut(data[var],4,retbins=True)
    data[var+'_'+'q'] = cut_series.astype(str)
data

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT,age_q,creatinine_phosphokinase_q,ejection_fraction_q,platelets_q,serum_creatinine_q,serum_sodium_q,time_q
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1,"(70.0, 95.0]","(250.0, 582.0]","(13.999, 30.0]","(262000.0, 303500.0]","(1.4, 9.4]","(112.999, 134.0]","(3.999, 73.0]"
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1,"(51.0, 60.0]","(582.0, 7861.0]","(30.0, 38.0]","(262000.0, 303500.0]","(0.9, 1.1]","(134.0, 137.0]","(3.999, 73.0]"
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1,"(60.0, 70.0]","(116.5, 250.0]","(13.999, 30.0]","(25099.999, 212500.0]","(1.1, 1.4]","(112.999, 134.0]","(3.999, 73.0]"
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1,"(39.999, 51.0]","(22.999, 116.5]","(13.999, 30.0]","(25099.999, 212500.0]","(1.4, 9.4]","(134.0, 137.0]","(3.999, 73.0]"
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1,"(60.0, 70.0]","(116.5, 250.0]","(13.999, 30.0]","(303500.0, 850000.0]","(1.4, 9.4]","(112.999, 134.0]","(3.999, 73.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0,"(60.0, 70.0]","(22.999, 116.5]","(30.0, 38.0]","(25099.999, 212500.0]","(0.9, 1.1]","(140.0, 148.0]","(203.0, 285.0]"
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0,"(51.0, 60.0]","(582.0, 7861.0]","(30.0, 38.0]","(262000.0, 303500.0]","(1.1, 1.4]","(137.0, 140.0]","(203.0, 285.0]"
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0,"(39.999, 51.0]","(582.0, 7861.0]","(45.0, 80.0]","(303500.0, 850000.0]","(0.499, 0.9]","(137.0, 140.0]","(203.0, 285.0]"
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0,"(39.999, 51.0]","(582.0, 7861.0]","(30.0, 38.0]","(25099.999, 212500.0]","(1.1, 1.4]","(137.0, 140.0]","(203.0, 285.0]"


In [29]:
## univariate analysis 
# The univariate analysis includes two outputs:
# Univariate table - in which the percentage of death is calculated for each of the variables categories.  
# pvalues_table - contains the P-values of each variable for predicting death.     

var_list = ['age_q','creatinine_phosphokinase_q','ejection_fraction_q','serum_creatinine_q','serum_sodium_q','time_q',
                  'anaemia','diabetes','high_blood_pressure','sex','smoking']  

univariate_table = pd.DataFrame(columns = ['var_name', 'var_cat', 'var_mean']) 
pvalues = []
for var in var_list:
    q_univariate = f""" select {var} as var_cat,
                              avg(death_event) as var_mean
                       from data
                       group by {var}"""

    univariate_table_ = pysqldf(q_univariate) 
    univariate_table_['var_name'] = [var]*univariate_table_.shape[0]
    univariate_table = pd.concat([univariate_table, univariate_table_])
    
    stat, p, dof, expected = chi2_contingency(pd.crosstab(index=data[var], columns=data['DEATH_EVENT']))
    pvalues.append(p)
    
univariate_table
pvalues_table = pd.DataFrame (pvalues, columns = ['pvalue'])
pvalues_table['var_name'] = var_list

In [30]:
univariate_table

,var_name,var_cat,var_mean
0,age_q,"(39.999, 51.0]",0.256410
1,age_q,"(51.0, 60.0]",0.285714
2,age_q,"(60.0, 70.0]",0.247059
3,age_q,"(70.0, 95.0]",0.596154
0,creatinine_phosphokinase_q,"(116.5, 250.0]",0.368421
1,creatinine_phosphokinase_q,"(22.999, 116.5]",0.253333
2,creatinine_phosphokinase_q,"(250.0, 582.0]",0.380952
3,creatinine_phosphokinase_q,"(582.0, 7861.0]",0.265625
0,ejection_fraction_q,"(13.999, 30.0]",0.548387
1,ejection_fraction_q,"(30.0, 38.0]",0.247191


In [6]:
pvalues_table

,pvalue,var_name
0,6.027344e-05,age_q
1,1.997368e-01,creatinine_phosphokinase_q
2,2.710166e-07,ejection_fraction_q
3,2.515347e-10,serum_creatinine_q
4,4.141172e-04,serum_sodium_q
5,4.846835e-27,time_q
6,3.073161e-01,anaemia
7,1.000000e+00,diabetes
8,2.141034e-01,high_blood_pressure
9,1.000000e+00,sex


In [5]:
# Preparing the X and y inputs for the model:
# X- a table of the explanatory variables which were found significant in the univariate analysis. 
# y - an output vector 
X = data.loc[:, data.columns != 'DEATH_EVENT']
y = data.loc[:, data.columns == 'DEATH_EVENT']

X=X[['age_q','ejection_fraction_q','serum_creatinine_q','serum_sodium_q','time_q']]
y = y.values.ravel() 

In [6]:
# Spliting X and y into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Applying Logistic regression on train data
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [7]:
# Calculating prediction accuracy on test data 
y_pred = logreg.predict(X_test)
metrics.accuracy_score(y_test, y_pred, normalize=True)

0.8333333333333334

In [8]:
###########################################################
### Building predictive model using Neural networks ###
###########################################################
## Initial model 
# Setting an architecture of the network
model = Sequential()
model.add(keras.layers.Dense(12, input_shape=(5,), activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [9]:
# Compiling the model
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [10]:
# Training the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
7/7 [==============================] - 1s 51ms/step - loss: 0.7633 - accuracy: 0.4641 - val_loss: 0.7736 - val_accuracy: 0.3778
Epoch 2/20
7/7 [==============================] - 0s 6ms/step - loss: 0.7269 - accuracy: 0.5072 - val_loss: 0.7394 - val_accuracy: 0.4667
Epoch 3/20
7/7 [==============================] - 0s 5ms/step - loss: 0.6957 - accuracy: 0.5598 - val_loss: 0.7098 - val_accuracy: 0.4778
Epoch 4/20
7/7 [==============================] - 0s 5ms/step - loss: 0.6710 - accuracy: 0.6077 - val_loss: 0.6822 - val_accuracy: 0.5444
Epoch 5/20
7/7 [==============================] - 0s 6ms/step - loss: 0.6468 - accuracy: 0.6459 - val_loss: 0.6590 - val_accuracy: 0.5778
Epoch 6/20
7/7 [==============================] - 0s 6ms/step - loss: 0.6268 - accuracy: 0.6699 - val_loss: 0.6382 - val_accuracy: 0.6778
Epoch 7/20
7/7 [==============================] - 0s 5ms/step - loss: 0.6093 - accuracy: 0.7177 - val_loss: 0.6203 - val_accuracy: 0.6667
Epoch 8/20
7/7 [=================

In [21]:
## Improving the initial model
# The accuracy of the initial model was much lower than the logistic regression model's accuracy (71.1% vs. 83.3%). 
# Therefore, I was trying to improve the model by hyperparameter search.
# The hyperparameters that were searched for were the number of layers and the learning rate.

def model_builder(hp):
  model = Sequential()
  model.add(InputLayer(input_shape=(5, ), name='Input_Layer'))
  
  # Tune the number of dense layers
  for i in range(hp.Int('num_layers', 1, 3)):
    
    # Tune the number of units in the each dense layer
    hp_units = hp.Int('units_'+str(i), min_value=1, max_value=50, step=10)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    
    # Tune the dropout rate in the each dense layer
        #hp_dropout = hp.Float('rate', min_value=0.0, max_value=0.5, step=0.1)
        #model.add(tf.keras.layers.Dropout(hp_dropout))
    
  # Add dense output layer
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  # Tune the learning rate for the optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [22]:
tuner = kt.RandomSearch(model_builder,
  objective='val_accuracy',
  max_trials=10,
  executions_per_trial=3,
  directory='keras_tuner',
  project_name='clinical_data_project',
  overwrite=True)

In [23]:
# Spliting X and y into train, validation and test:
# At the first step, X and y are splitted into train and test
# At the second step, train data is splitted into train and validation for the hyperparameters search process
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
tuner.search(X_train, y_train, epochs=20, validation_split=0.2)

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.6481481393178304

Best val_accuracy So Far: 0.8765432238578796
Total elapsed time: 00h 00m 48s
INFO:tensorflow:Oracle triggered exit


In [24]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

{'num_layers': 2, 'units_0': 21, 'learning_rate': 0.1, 'units_1': 41, 'units_2': 21}


In [29]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20
7/7 [==============================] - 1s 31ms/step - loss: 0.6351 - accuracy: 0.7860 - val_loss: 0.4714 - val_accuracy: 0.7778
Epoch 2/20
7/7 [==============================] - 0s 6ms/step - loss: 0.3998 - accuracy: 0.8326 - val_loss: 0.4867 - val_accuracy: 0.8519
Epoch 3/20
7/7 [==============================] - 0s 6ms/step - loss: 0.3615 - accuracy: 0.8279 - val_loss: 0.4242 - val_accuracy: 0.8333
Epoch 4/20
7/7 [==============================] - 0s 6ms/step - loss: 0.3528 - accuracy: 0.8419 - val_loss: 0.5013 - val_accuracy: 0.8148
Epoch 5/20
7/7 [==============================] - 0s 6ms/step - loss: 0.3253 - accuracy: 0.8651 - val_loss: 0.4415 - val_accuracy: 0.8519
Epoch 6/20
7/7 [==============================] - 0s 5ms/step - loss: 0.3175 - accuracy: 0.8651 - val_loss: 0.6117 - val_accuracy: 0.7963
Epoch 7/20
7/7 [==============================] - 0s 5ms/step - loss: 0.3198 - accuracy: 0.8558 - val_loss: 0.4587 - val_accuracy: 0.8519
Epoch 8/20
7/7 [=================

In [30]:
# Calculate prediction accuracy on the test data
model.evaluate(X_test, y_test, return_dict=True)

1/1 [==============================] - 0s 22ms/step - loss: 0.3874 - accuracy: 0.8333


{'loss': 0.3874319791793823, 'accuracy': 0.8333333134651184}

In [12]:
###########################################################
### Building predictive model using Random Forest ###
###########################################################

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model=RandomForestClassifier(n_estimators=1000,max_features='sqrt')
fitted_model=model.fit(X_train,y_train) 
predictions=fitted_model.predict(X_test)
metrics.accuracy_score(y_test, predictions, normalize=True)


0.8

In [35]:
random_forest_model = RandomForestClassifier(n_jobs=-1,max_features='sqrt')
param_grid= {"n_estimators": [10,100,500,1000],
             "max_depth": [1,5,10,15],
             "min_samples_leaf": [1,2,4,10,15,30,50]}
#grid_search = GridSearchCV(estimator=random_forest_model,param_grid=param_grid,cv=10)
#grid_search.fit(X_train,y_train)
#print(grid_search.best_params_)
search = HalvingRandomSearchCV(random_forest_model, param_grid,
                                resource='n_samples',
                                max_resources='auto',
                                random_state=0).fit(X_train, y_train)


In [36]:
print(search.best_params_)
predictions = search.predict(X_test)
print(metrics.accuracy_score(y_test, predictions))

{'n_estimators': 100, 'min_samples_leaf': 2, 'max_depth': 5}
0.8666666666666667


TypeError: 'tuple' object cannot be interpreted as an integer